In [20]:
import pandas as pd
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
import re
import json
import csv 

%matplotlib inline


In [38]:
!pip install emoji==0.6.0


     |████████████████████████████████| 51kB 1.5MB/s 
  Created wheel for emoji: filename=emoji-0.6.0-cp37-none-any.whl size=49716 sha256=9947af045f27be9e27c47ca96685157a64a48680765be160c788f985faeb473c
  Stored in directory: /root/.cache/pip/wheels/46/2c/8b/9dcf5216ca68e14e0320e283692dce8ae321cdc01e73e17796
Successfully built emoji


In [39]:
import emoji


In [11]:
with open('/content/drive/MyDrive/Omdena_sentiment/Dataset/labr.json') as file:
    data = json.load(file)

In [27]:
with open("/content/drive/MyDrive/Omdena_sentiment/Dataset/labr.csv", "w") as file:
    csv_file = csv.writer(file)
    csv_file.writerow(['Class','Text'])  # header row
    for item in data:
        
        csv_file.writerow([data[item]['Class'], data[item]['Text']])

In [56]:
df=pd.read_csv('/content/drive/MyDrive/Omdena_sentiment/Dataset/labr.csv')

In [57]:
df.head(10)

,Class,Text
0,4,"""عزازيل الذي صنعناه ،الكامن في أنفسنا"" يذكرني..."
1,4,من أمتع ما قرأت من روايات بلا شك. وحول الشك ت...
2,4,رواية تتخذ من التاريخ ،جوًا لها اختار المؤلف ...
3,1,إني أقدّر هذه الرواية كثيرا، لسبب مختلف عن أس...
4,5,الكاهن الذي أطلق على نفسه اسم هيبا تيمنا بالع...
5,5,"""عزازيل"" هو اسم رواية يوسف زيدان الثانية و ال..."
6,3,لم استوعب بداية أنها رواية حتى وصلت إلى المنت...
7,4,هذا هو الأدب . هذه هي الرواية !
8,4,مقتطفات من رواية عزازيل للكاتب يوسف زيدان ---...
9,5,في البداية.ولكن بداية مشوقة للغاية.يبدو انني ...


In [58]:
def clean_text(text):  
    # ref: https://github.com/bakrianoo/aravec
    search = ["أ","إ","آ","ة","_","-","/",".","،"," و "," يا ",'"',"ـ","'","ى",
              "\\",'\n', '\t','&quot;','?','؟','!']
    replace = ["ا","ا","ا","ه"," "," ","","",""," و"," يا",
               "","","","ي","",' ', ' ',' ',' ? ',' ؟ ', ' ! ']
    
    tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    text = re.sub(tashkeel,"", text)
    
    # longation = re.compile(r'(.)\1+')
    # subst = r"\1\1"
    # text = re.sub(longation, subst, text)
    
    text = re.sub(r"[^\w\s]", '', text)
    text = re.sub(r"[a-zA-Z]", '', text)
    text = re.sub(r"\d+", ' ', text)
    text = re.sub(r"\n+", ' ', text)
    text = re.sub(r"\t+", ' ', text)
    text = re.sub(r"\r+", ' ', text)
    text = re.sub(r"\s+", ' ', text)
    text = text.replace('وو', 'و')
    text = text.replace('يي', 'ي')
    text = text.replace('اا', 'ا')
    
    for i in range(0, len(search)):
        text = text.replace(search[i], replace[i])
    
    text = text.strip()
    
    return text

In [59]:
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

In [60]:
df['cleaned_text'] = df.Text.apply(clean_text)
df["word_count"] = df["cleaned_text"].apply(lambda x: len(x.split()))

In [61]:
df.head()

,Class,Text,cleaned_text,word_count
0,4,"""عزازيل الذي صنعناه ،الكامن في أنفسنا"" يذكرني...",عزازيل الذي صنعناه الكامن في انفسنا يذكرني يوس...,101
1,4,من أمتع ما قرأت من روايات بلا شك. وحول الشك ت...,من امتع ما قرات من روايات بلا شك وحول الشك تدن...,16
2,4,رواية تتخذ من التاريخ ،جوًا لها اختار المؤلف ...,روايه تتخذ من التاريخ جوا لها اختار المؤلف فتر...,31
3,1,إني أقدّر هذه الرواية كثيرا، لسبب مختلف عن أس...,اني اقدر هذه الروايه كثيرا لسبب مختلف عن اسباب...,498
4,5,الكاهن الذي أطلق على نفسه اسم هيبا تيمنا بالع...,الكاهن الذي اطلق علي نفسه اسم هيبا تيمنا بالعا...,124


In [65]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['Text'], df['Class'], test_size=0.2, random_state=1)



In [67]:
 X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1) # 0.25 x 0.8 = 0.2

In [73]:
series_X_Train = pd.Series(X_train)
series_Y_Train = pd.Series(y_train)

series_X_Test = pd.Series(X_test)
series_Y_Test = pd.Series(y_test)


series_X_val = pd.Series(X_val)
series_Y_val = pd.Series(y_val)



In [74]:
df_X_train = pd.DataFrame(series_X_Train)
df_X_train = df_X_train.rename(columns = {0:'Text'})

df_y_train = pd.DataFrame(series_Y_Train)
df_y_train = df_y_train.rename(columns = {0:'Class'})

df_all = pd.concat([df_X_train, df_y_train], axis=1)


In [75]:
df_all.head()

,Text,Class
54225,رائعة جدا العطر ( قصة قاتل للمبدع الألماني با...,5
24320,قرأت أرز باللبن لشخصين عشر مرات قبل تكملة الك...,4
5866,ثم شرد من جديد كأنه يفكر في شئ آخر و قال بشئ ...,4
35391,الرواية ممتعة ولكن بسبب طولها اجلتها لفترة وج...,3
56519,كنت اول مرة اقرأ عن المسرح اللا معقول,5


In [79]:
df_all.to_csv('/content/drive/MyDrive/Omdena_sentiment/Dataset/train_labr.csv')

In [77]:
df_X_val = pd.DataFrame(series_X_val)
df_X_val = df_X_val.rename(columns = {0:'Text'})

df_y_val = pd.DataFrame(series_Y_val)
df_y_val = df_y_val.rename(columns = {0:'Class'})

df_val_all = pd.concat([df_X_val, df_y_val], axis=1)


In [78]:
df_val_all.head(10)

,Text,Class
7199,كتاب بالف كتاب,5
27988,يقدم الكتاب رؤية متصلة لعلم الكلام ويرينا امت...,5
10440,عشت لحظات جميلة مع هذا الكتاب حيث نقلنا من خل...,4
6008,الأجداد لا بأس . أما الأحفاد فلا يصلحون إلا ل...,5
58089,الكتاب يتحدث عن حياة شخص يدعي بيتر هوركوس. وا...,1
49671,بلاغة لا حدود لها من كلام أمير المؤمنين رضي ا...,5
37312,اشعر اننى اقل من ان ابدى رأيا فى احلى نزاريات...,5
14681,عجبتنى جدا مقالة زغروطة وحشة . و مقال قصيدة ع...,2
24826,كتاب يعلو بروحك الى السماء فترفرف عاليا بجانب...,5
11424,جميييييييييييييييييييل :),4


In [80]:
df_val_all.to_csv('/content/drive/MyDrive/Omdena_sentiment/Dataset/val_labr.csv')

In [81]:
df_X_test = pd.DataFrame(series_X_Test)
df_X_test = df_X_test.rename(columns = {0:'Text'})

df_y_test = pd.DataFrame(series_Y_Test)
df_y_test = df_y_test.rename(columns = {0:'Class'})

df_test_all = pd.concat([df_X_test, df_y_test], axis=1)


In [82]:
df_test_all.head()

,Text,Class
33728,أرى أن الأذواق تختلف و لا تقبل أي نعوت بالرذا...,4
22401,"عندما بدات في قراءة هذا الكتاب, اعتقدت خطأ ان...",1
21273,رواية جميله جدا و لغة الكاتب مبهره(مع تحفظي ع...,5
10447,اختير كأفضل كتاب سيرة للمصطفى محمد حبيبنا صلى...,4
44752,رحمك الله ياسيد، كم أثريتنا وكم عريتهم!,5


In [83]:
df_test_all.to_csv('/content/drive/MyDrive/Omdena_sentiment/Dataset/test_labr.csv')